<a href="https://colab.research.google.com/github/Pakhi27/Text-Summarization--Amazon-Review-/blob/main/Text_Summarization_Email.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [3]:
!pip show huggingface_hub

Name: huggingface-hub
Version: 0.23.5
Summary: Client library to download and publish models, datasets and other repos on the huggingface.co hub
Home-page: https://github.com/huggingface/huggingface_hub
Author: Hugging Face, Inc.
Author-email: julien@huggingface.co
License: Apache
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, fsspec, packaging, pyyaml, requests, tqdm, typing-extensions
Required-by: accelerate, datasets, tokenizers, transformers


In [4]:
pip install --upgrade huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.5/417.5 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.23.5
    Uninstalling huggingface-hub-0.23.5:
      Successfully uninstalled huggingface-hub-0.23.5


In [ ]:
from huggingface_hub import login

login()


In [5]:
from datasets import load_dataset

dataset = load_dataset("cnn_dailymail", "3.0.0")
print(dataset)
# Description: This dataset contains news articles from CNN and DailyMail with associated human-written summaries. It’s widely used for training and evaluating text summarization models.



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})


In [8]:
# Import necessary libraries
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import pipeline

In [9]:
import pandas as pd

In [10]:
# Split dataset into training and validation sets
train_dataset = pd.DataFrame(dataset["train"])
val_dataset = pd.DataFrame(dataset["validation"])
eval_dataset = dataset["test"]

# Save the datasets to CSV files
train_dataset.to_csv("train.csv", index=False)
val_dataset.to_csv("val.csv", index=False)
eval_dataset.to_csv("eval.csv", index=False)

Creating CSV from Arrow format:   0%|          | 0/12 [00:00<?, ?ba/s]

49890690

In [11]:
print(train_dataset)

                                                  article  \
0       LONDON, England (Reuters) -- Harry Potter star...   
1       Editor's note: In our Behind the Scenes series...   
2       MINNEAPOLIS, Minnesota (CNN) -- Drivers who we...   
3       WASHINGTON (CNN) -- Doctors removed five small...   
4       (CNN)  -- The National Football League has ind...   
...                                                   ...   
287108  The nine-year-old daughter of a black, unarmed...   
287109  Legalising assisted suicide is a slippery slop...   
287110  A group calling itself 'The Women of the 99 Pe...   
287111  Most men enjoy a good pint of lager or real al...   
287112  A Facebook page seeking to preserve the 'Black...   

                                               highlights  \
0       Harry Potter star Daniel Radcliffe gets £20M f...   
1       Mentally ill inmates in Miami are housed on th...   
2       NEW: "I thought I was going to die," driver sa...   
3       Five small poly

In summary, the model used is BART, and specifically the variant that is pre-trained and fine-tuned by Facebook AI on the CNN/DailyMail dataset, which is a standard for summarization tasks.ChatGPT
The model used in the above text summarization example is BART (Bidirectional and Auto-Regressive Transformers), specifically the facebook/bart-large-cnn variant.

About BART:
Architecture: BART is a sequence-to-sequence model with a bidirectional encoder (like BERT) and an autoregressive decoder (like GPT). This combination makes it particularly effective for tasks that involve generating text, such as summarization, translation, and more.

Pre-training: BART is pre-trained on a denoising autoencoder task, where parts of the input text are corrupted, and the model is trained to reconstruct the original text. This helps the model learn a strong understanding of language and how to generate coherent text.

Fine-tuning: The specific variant used, facebook/bart-large-cnn, is fine-tuned on the CNN/DailyMail dataset for the task of abstractive summarization. It has been trained to generate concise summaries of news articles.

Why BART for Summarization?
Strong Performance: BART has been shown to achieve state-of-the-art results on various summarization benchmarks, including the CNN/DailyMail dataset.
Flexibility: Its sequence-to-sequence nature allows it to generate high-quality abstractive summaries, which means it can produce summaries that are not just extracted sentences from the text but rephrased and condensed versions of the input.
Key Features:
Encoder-Decoder Structure: The encoder processes the entire input sequence bidirectionally, providing a comprehensive understanding of the text. The decoder then generates the summary autoregressively, predicting one word at a time while considering the previously generated words.
Pre-trained and Fine-tuned: The facebook/bart-large-cnn model has 400 million parameters and is fine-tuned specifically on summarization tasks, making it well-suited for generating high-quality summaries.


In [20]:
# Load the tokenizer
# The articles and highlights (summaries) are tokenized to prepare the data for the model.
# In summary, the model used is BART, and specifically the variant that is pre-trained and fine-tuned by Facebook AI on the CNN/DailyMail dataset, which is a standard for summarization tasks.

from transformers import AutoTokenizer
from datasets import load_dataset

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")

# Load the dataset
dataset = load_dataset("cnn_dailymail", "3.0.0")

# Define the number of samples you want to select
num_samples = 300  # Change this to the desired number of samples

# Sample a subset of each split in the dataset
sampled_datasets = {}
for split, data in dataset.items():
    sampled_datasets[split] = data.select(range(min(num_samples, len(data))))

# Define the preprocessing function
def preprocess_function(examples):
    inputs = [doc for doc in examples['article']]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["highlights"], max_length=150, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply the preprocessing function to the sampled datasets
tokenized_datasets = {split: sampled_datasets[split].map(preprocess_function, batched=True) for split in sampled_datasets}




Map:   0%|          | 0/300 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4016: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [21]:
# Load the pre-trained model
# Model Loading:
# The BART model facebook/bart-large-cnn is loaded, which is pre-trained for summarization tasks.
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-base")


In [22]:
# Set up training arguments
# Training Arguments:
# Seq2SeqTrainingArguments is configured with settings like learning rate, batch size, number of epochs, etc.
# Training:
# The model is trained on the dataset using Seq2SeqTrainer.
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=1,
    predict_with_generate=True,
    logging_dir='./logs',
)

# Data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [23]:
!pip install rouge-score


In [18]:
# Load the ROUGE metric
# Evaluation:
# The trained model is evaluated on the validation set, and the ROUGE metric is computed to assess the quality of the summaries.
rouge_metric = load_metric("rouge")

# Define a function for ROUGE computation
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # ROUGE expects a newline after each sentence
    decoded_preds = ["\n".join(pred.strip() for pred in pred.splitlines()) for pred in decoded_preds]
    decoded_labels = ["\n".join(label.strip() for label in label.splitlines()) for label in decoded_labels]

    result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    return result


<ipython-input-18-0c85b71d3eef>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric("rouge")


The repository for rouge contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/rouge.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


In [24]:
# Initialize the trainer
# Number of samples to use for training and evaluation
train_sample_size = 300  # Adjust this number based on your system's capacity
eval_sample_size = 200    # Adjust this number based on your system's capacity

# Select a subset of the training dataset
small_train_dataset = tokenized_datasets["train"].select(range(train_sample_size))

# Select a subset of the validation dataset
small_eval_dataset = tokenized_datasets["validation"].select(range(eval_sample_size))

# Initialize the trainer with the smaller dataset
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


In [26]:

# Train the model
trainer.train()


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,No log,0.909396,23.935117,9.487025,19.665915,22.135363


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=150, training_loss=1.8238787841796875, metrics={'train_runtime': 3394.0767, 'train_samples_per_second': 0.088, 'train_steps_per_second': 0.044, 'total_flos': 91460468736000.0, 'train_loss': 1.8238787841796875, 'epoch': 1.0})

In [27]:
# Evaluate the model
eval_results = trainer.evaluate()
print(f"Evaluation Results: {eval_results}")

Evaluation Results: {'eval_loss': 0.909396231174469, 'eval_rouge1': 23.935117383274086, 'eval_rouge2': 9.487024727894799, 'eval_rougeL': 19.66591456605088, 'eval_rougeLsum': 22.135362912631887, 'eval_runtime': 1066.4737, 'eval_samples_per_second': 0.188, 'eval_steps_per_second': 0.094, 'epoch': 1.0}


In [28]:
# Generating summaries for test dataset
# Summarization Pipeline:
# A text summarization pipeline is created to generate summaries from the test dataset and compare them with reference summaries.
summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)

In [29]:
for i in range(3):  # Change range to see more summaries
    article = dataset["test"][i]["article"]
    reference_summary = dataset["test"][i]["highlights"]

    generated_summary = summarizer(article, max_length=150, min_length=40, do_sample=False)[0]['summary_text']

    print(f"\nArticle: {article[:500]}...")
    print(f"\nReference Summary: {reference_summary}")
    print(f"\nGenerated Summary: {generated_summary}")
# ROUGE Score:
# The ROUGE score is used to measure the quality of the generated summaries in comparison to the reference summaries. The final scores are printed after evaluation.


Article: (CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC's founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, includin...

Reference Summary: Membership gives the ICC jurisdiction over alleged crimes committed in Palestinian territories since last June .
Israel and the United States opposed the move, which could open the door to war crimes investigations against Israelis .

Generated Summary: Palestinian Authority officially becomes 123rd member of the International Criminal Court .
Palestinian Authority formally becomes a State Party to the Rome Statute .
Israel and the United States oppose Palestini